В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [367]:
from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import lightgbm as lgb


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

In [368]:
raw_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-2/train.csv")

cat_cols = raw_df.select_dtypes('object').columns
raw_df[cat_cols] = raw_df[cat_cols].astype('category')

In [369]:
X_cols = raw_df.drop(columns=['id', 'CustomerId', 'Exited']).columns
y_col = raw_df[['Exited']].columns[0]

train_df, test_df = split_train_val(raw_df, y_col)
(X_train, y_train), (X_test, y_test) = separate_inputs_targets(train_df, X_cols, y_col), separate_inputs_targets(test_df, X_cols, y_col)

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [370]:
# did it above for full dataset at once

2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [371]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12000 entries, 7180 to 9360
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Surname          12000 non-null  category
 1   CreditScore      12000 non-null  float64 
 2   Geography        12000 non-null  category
 3   Gender           12000 non-null  category
 4   Age              12000 non-null  float64 
 5   Tenure           12000 non-null  float64 
 6   Balance          12000 non-null  float64 
 7   NumOfProducts    12000 non-null  float64 
 8   HasCrCard        12000 non-null  float64 
 9   IsActiveMember   12000 non-null  float64 
 10  EstimatedSalary  12000 non-null  float64 
dtypes: category(3), float64(8)
memory usage: 913.0 KB


In [372]:
xgb_model = xgb.XGBClassifier(
    tree_method="hist",
    eval_metric="auc",
    missing=np.nan,
    enable_categorical=True,
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    random_state=42,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_weight=10,
    gamma=0.1,
    alpha=0.1,
)

xgb_model.fit(X_train, y_train)

train_preds = xgb_model.predict_proba(X_train)[:, 1]
val_preds = xgb_model.predict_proba(X_test)[:, 1]

train_auc = roc_auc_score(y_train, train_preds)
val_auc = roc_auc_score(y_test, val_preds)

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

Train AUROC: 0.9427
Validation AUROC: 0.9380


**Висновок**
- Висока точність на обох наборах свідчить про те, що модель добре навчається і не страждає від явного high bias або high variance.
- XGBoost показала дуже хороші результати, порівняно з Decision Tree, що підтверджує її кращу здатність до узагальнення та точності

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [373]:
def objective_xgb(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    model = xgb.XGBClassifier(
        tree_method="hist",
        eval_metric="auc",
        missing=np.nan,
        enable_categorical=True,
        random_state=42,
        **params
    )
    
    model.fit(X_train, y_train)
    
    train_preds = model.predict_proba(X_train)[:, 1]
    val_preds = model.predict_proba(X_test)[:, 1]
    
    train_auc = roc_auc_score(y_train, train_preds)
    test_auc = roc_auc_score(y_test, val_preds)
    
    return {'loss': -test_auc, 'status': STATUS_OK}

In [ ]:
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 100), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.5),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'gamma': hp.uniform('gamma', 0, 1),
    'alpha': hp.uniform('alpha', 0, 1)
}

trials = Trials()
best = fmin(fn=objective_xgb, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

100%|██████████| 20/20 [00:23<00:00,  1.19s/trial, best loss: -0.9382989231085809]
{'alpha': 0.65,
 'colsample_bytree': 0.59,
 'gamma': 0.73,
 'learning_rate': 0.01,
 'max_depth': 5,
 'min_child_weight': 3,
 'n_estimators': 800,
 'subsample': 0.83}


In [385]:
for i, v in best.items():
    best[i] = int(v) if v > 1 else round(float(v), 2)

pprint(best)

{'colsample_bytree': 0.82,
 'learning_rate': 0.05,
 'max_depth': 1.0,
 'min_child_weight': 9,
 'n_estimators': 400,
 'num_leaves': 101,
 'reg_alpha': 0.51,
 'reg_lambda': 0.8,
 'subsample': 0.76}


In [375]:
final_xgb_clf = xgb.XGBClassifier(
    tree_method="hist",
    eval_metric="auc",
    missing=np.nan,
    enable_categorical=True,
    random_state=42,
    n_estimators=best['n_estimators'],
    learning_rate=best['learning_rate'],
    max_depth=best['max_depth'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    min_child_weight=best['min_child_weight'],
    gamma=best['gamma'],
    alpha=best['alpha']
)

final_xgb_clf.fit(X_train, y_train)

train_preds = final_xgb_clf.predict_proba(X_train)[:, 1]
test_preds = final_xgb_clf.predict_proba(X_test)[:, 1]

train_auc = roc_auc_score(y_train, train_preds)
test_auc = roc_auc_score(y_test, test_preds)

print("Final model Train AUROC:", train_auc)
print("Final model Validation AUROC:", val_auc)

Final model Train AUROC: 0.9570876303456342
Final model Validation AUROC: 0.9379583647712464


**Висновок**
- Модель хороша, оскільки AUROC на тренувальних та валідаційних наборах дуже високий, і це вказує на добре налаштовану модель з високою точністю.
- Ці результати дуже схожі на попередні (після першого навчання моделі), де Train AUROC був 0.9427 і Validation AUROC 0.9380.

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [376]:
cat_features = ['Surname', 'Geography', 'Gender']
cat_features = [X_train.columns.get_loc(col) for col in cat_features]

lgbm_model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=3,
    random_state=42,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_weight=10,
    alpha=0.1,
    cat_feature=cat_features
)

lgbm_model.fit(X_train, y_train)

train_preds = lgbm_model.predict_proba(X_train)[:, 1]
test_preds = lgbm_model.predict_proba(X_test)[:, 1]

train_auc = roc_auc_score(y_train, train_preds)
test_auc = roc_auc_score(y_test, test_preds)

print(f"Train AUROC: {train_auc}")
print(f"Validation AUROC: {test_auc}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] categorical_feature is set with cat_feature=0,2,3, categorical_column=0,2,3 will be ignored. Current value: categorical_feature=0,2,3
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001649 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1316
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [377]:
def objective_lgb(params):
    model = lgb.LGBMClassifier(
        learning_rate=params['learning_rate'],
        num_leaves=int(params['num_leaves']),
        max_depth=int(params['max_depth']),
        n_estimators=int(params['n_estimators']),
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        min_child_weight=params['min_child_weight'],
        random_state=42
    )
    
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='auc')
    
    test_preds = model.predict_proba(X_test)[:, 1]
    
    auc = roc_auc_score(y_test, test_preds)
    return -auc

In [ ]:
space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'num_leaves': hp.quniform('num_leaves', 31, 128, 1),
    'max_depth': hp.choice('max_depth', [-1, 3, 5, 7, 9, 11]),
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
}

best = fmin(fn=objective_lgb, space=space, algo=tpe.suggest, max_evals=10)

final_lgb_clf = lgb.LGBMClassifier(
    learning_rate=best['learning_rate'],
    num_leaves=int(best['num_leaves']),
    max_depth=int(best['max_depth']),
    n_estimators=int(best['n_estimators']),
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    min_child_weight=best['min_child_weight'],
    random_state=42
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000714 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1316                     
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561 
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

In [384]:
for i, v in best.items():
    best[i] = int(v) if v > 1 else round(float(v), 2)

pprint(best)

{'colsample_bytree': 0.82,
 'learning_rate': 0.05,
 'max_depth': 1.0,
 'min_child_weight': 9,
 'n_estimators': 400,
 'num_leaves': 101,
 'reg_alpha': 0.51,
 'reg_lambda': 0.8,
 'subsample': 0.76}


In [379]:
final_lgb_clf.fit(X_train, y_train)

train_preds = final_lgb_clf.predict_proba(X_train)[:, 1]
test_preds = final_lgb_clf.predict_proba(X_test)[:, 1]

train_auc = roc_auc_score(y_train, train_preds)
test_auc = roc_auc_score(y_test, test_preds)

print(f"Train AUROC: {train_auc}")
print(f"Validation AUROC: {test_auc}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000672 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1316
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

In [380]:
test_raw_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-2/test.csv")

ids = test_raw_df["id"].values

test_raw_df = test_raw_df.drop(['id', 'CustomerId'], axis=1)
test_raw_df[cat_cols] = test_raw_df[cat_cols].astype('category')

prediction_probs = final_lgb_clf.predict_proba(test_raw_df)[:, 1]

# Формування submission.csv
sample_raw_df = pd.DataFrame({'id': ids})
sample_raw_df['Exited'] = prediction_probs
sample_raw_df.to_csv("bank-customer-churn-prediction-dlu-course-c-2/submission_log_reg.csv", index=False)